In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras

#from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import pylab
#plt.rcParams['figure.figsize'] = 10, 10
#%matplotlib inline

Using TensorFlow backend.


In [2]:
train = pd.read_json("data/processed/train.json")
target_train=train['is_iceberg']
test = pd.read_json("data/processed/test.json")

target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [3]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#X_band_3=(X_band_1+X_band_2)/2
X_band_3=np.fabs(np.subtract(X_band_1,X_band_2))
X_band_4=np.maximum(X_band_1,X_band_2)
X_band_5=np.minimum(X_band_1,X_band_2)
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([                       
                          X_band_3[:, :, :, np.newaxis],X_band_4[:, :, :, np.newaxis],X_band_5[:, :, :, np.newaxis]], axis=-1)

In [4]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
#X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_band_test_3=np.fabs(np.subtract(X_band_test_1,X_band_test_2))
X_band_test_4=np.maximum(X_band_test_1,X_band_test_2)
X_band_test_5=np.minimum(X_band_test_1,X_band_test_2)
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([
                          X_band_test_3[:, :, :, np.newaxis], X_band_test_4[:, :, :, np.newaxis],X_band_test_5[:, :, :, np.newaxis]],axis=-1)

In [5]:
from tqdm import tqdm
from skimage.transform import resize

def resize_images(df, width):
    n = len(df)
    df_resized = np.zeros((n, width, width, 3), dtype=np.float32)
    for i in tqdm(range(n)):
        x = df[i]
        x = (x-x.min())/(x.max()-x.min()) # normalize for each pseudo pixel value
        df_resized[i] = resize(x, (width,width), mode='reflect')
    return df_resized

In [6]:
X_train = resize_images(X_train, 224)
X_test = resize_images(X_test, 224)

100%|██████████████████████████████████████████████████████████████████████████████| 8424/8424 [01:25<00:00, 98.63it/s]


In [7]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator

In [8]:
batch_size=16
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0,
                         height_shift_range = 0,
                         channel_shift_range=0,
                         zoom_range = 0.5,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

In [9]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [msave, es, reduce_lr_loss]

In [10]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    #base_model = VGG16(weights='imagenet', include_top=False, 
                 #input_shape=X_train.shape[1:], classes=1)
    base_model = VGG19(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    x = GlobalMaxPooling2D()(x)
    base_model2 = MobileNet(weights=None, alpha=1,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

    x2 = base_model2.output
    x2 = GlobalAveragePooling2D()(x2)

    merge_one = concatenate([x, x2, angle_layer])

    merge_one = Dropout(0.6)(merge_one)
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [11]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [12]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


E:\software\anaconda\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - ETA: 5:39 - loss: 0.7346 - acc: 0.500 - ETA: 2:49 - loss: 0.7149 - acc: 0.468 - ETA: 1:51 - loss: 0.7318 - acc: 0.437 - ETA: 1:22 - loss: 0.7326 - acc: 0.453 - ETA: 1:04 - loss: 0.7395 - acc: 0.462 - ETA: 52s - loss: 0.7722 - acc: 0.447 - ETA: 43s - loss: 0.7661 - acc: 0.46 - ETA: 36s - loss: 0.7925 - acc: 0.44 - ETA: 31s - loss: 0.8049 - acc: 0.44 - ETA: 27s - loss: 0.8047 - acc: 0.43 - ETA: 23s - loss: 0.8069 - acc: 0.44 - ETA: 20s - loss: 0.8146 - acc: 0.43 - ETA: 17s - loss: 0.8079 - acc: 0.44 - ETA: 15s - loss: 0.7988 - acc: 0.44 - ETA: 13s - loss: 0.7963 - acc: 0.44 - ETA: 11s - loss: 0.7916 - acc: 0.44 - ETA: 9s - loss: 0.7991 - acc: 0.4412 - ETA: 7s - loss: 0.8026 - acc: 0.437 - ETA: 6s - loss: 0.8085 - acc: 0.440 - ETA: 4s - loss: 0.8043 - acc: 0.443 - ETA: 3s - loss: 0.7990 - acc: 0.446 - ETA: 2s - loss: 0.7950 - acc: 0.457 - ETA: 1s - loss: 0.7941 - acc: 0.456 - 37s 2s/step - loss: 0.7947 - acc: 0.4557 - val_loss: 0.7014 -

24/24 [==============================] - ETA: 11s - loss: 0.2313 - acc: 1.00 - ETA: 10s - loss: 0.3719 - acc: 0.81 - ETA: 10s - loss: 0.3938 - acc: 0.81 - ETA: 9s - loss: 0.3777 - acc: 0.8125 - ETA: 9s - loss: 0.3931 - acc: 0.825 - ETA: 9s - loss: 0.3876 - acc: 0.822 - ETA: 8s - loss: 0.3771 - acc: 0.839 - ETA: 8s - loss: 0.3859 - acc: 0.835 - ETA: 7s - loss: 0.3793 - acc: 0.833 - ETA: 7s - loss: 0.3946 - acc: 0.825 - ETA: 6s - loss: 0.4220 - acc: 0.801 - ETA: 6s - loss: 0.4349 - acc: 0.791 - ETA: 5s - loss: 0.4342 - acc: 0.793 - ETA: 5s - loss: 0.4324 - acc: 0.799 - ETA: 4s - loss: 0.4201 - acc: 0.804 - ETA: 4s - loss: 0.4206 - acc: 0.804 - ETA: 3s - loss: 0.4222 - acc: 0.801 - ETA: 3s - loss: 0.4193 - acc: 0.802 - ETA: 2s - loss: 0.4248 - acc: 0.799 - ETA: 2s - loss: 0.4209 - acc: 0.803 - ETA: 1s - loss: 0.4157 - acc: 0.812 - ETA: 0s - loss: 0.4343 - acc: 0.801 - ETA: 0s - loss: 0.4290 - acc: 0.804 - 14s 599ms/step - loss: 0.4302 - acc: 0.8018 - val_loss: 0.4017 - val_acc: 0.8292
Epo

24/24 [==============================] - ETA: 11s - loss: 0.4333 - acc: 0.81 - ETA: 10s - loss: 0.4202 - acc: 0.81 - ETA: 10s - loss: 0.3927 - acc: 0.81 - ETA: 10s - loss: 0.3807 - acc: 0.81 - ETA: 9s - loss: 0.4135 - acc: 0.7750 - ETA: 9s - loss: 0.4155 - acc: 0.791 - ETA: 8s - loss: 0.3850 - acc: 0.812 - ETA: 8s - loss: 0.3806 - acc: 0.804 - ETA: 7s - loss: 0.3843 - acc: 0.798 - ETA: 7s - loss: 0.3810 - acc: 0.806 - ETA: 6s - loss: 0.3920 - acc: 0.806 - ETA: 6s - loss: 0.3828 - acc: 0.812 - ETA: 5s - loss: 0.3819 - acc: 0.817 - ETA: 5s - loss: 0.3837 - acc: 0.817 - ETA: 4s - loss: 0.3818 - acc: 0.816 - ETA: 4s - loss: 0.3840 - acc: 0.808 - ETA: 3s - loss: 0.3931 - acc: 0.805 - ETA: 3s - loss: 0.4121 - acc: 0.798 - ETA: 2s - loss: 0.4144 - acc: 0.796 - ETA: 2s - loss: 0.4088 - acc: 0.800 - ETA: 1s - loss: 0.4171 - acc: 0.791 - ETA: 1s - loss: 0.4093 - acc: 0.798 - ETA: 0s - loss: 0.4018 - acc: 0.801 - 18s 763ms/step - loss: 0.4005 - acc: 0.7995 - val_loss: 0.3449 - val_acc: 0.8727
Epo

...

24/24 [==============================] - ETA: 10s - loss: 0.2774 - acc: 0.87 - ETA: 10s - loss: 0.2352 - acc: 0.90 - ETA: 10s - loss: 0.2167 - acc: 0.91 - ETA: 9s - loss: 0.2440 - acc: 0.8906 - ETA: 9s - loss: 0.2443 - acc: 0.900 - ETA: 8s - loss: 0.2779 - acc: 0.895 - ETA: 8s - loss: 0.2802 - acc: 0.901 - ETA: 7s - loss: 0.2896 - acc: 0.890 - ETA: 7s - loss: 0.2887 - acc: 0.881 - ETA: 6s - loss: 0.2822 - acc: 0.881 - ETA: 6s - loss: 0.2939 - acc: 0.880 - ETA: 5s - loss: 0.3069 - acc: 0.875 - ETA: 5s - loss: 0.3088 - acc: 0.870 - ETA: 4s - loss: 0.3027 - acc: 0.870 - ETA: 4s - loss: 0.3001 - acc: 0.870 - ETA: 3s - loss: 0.3179 - acc: 0.859 - ETA: 3s - loss: 0.3111 - acc: 0.860 - ETA: 2s - loss: 0.3216 - acc: 0.857 - ETA: 2s - loss: 0.3349 - acc: 0.855 - ETA: 1s - loss: 0.3415 - acc: 0.850 - ETA: 1s - loss: 0.3308 - acc: 0.857 - ETA: 0s - loss: 0.3261 - acc: 0.860 - ETA: 0s - loss: 0.3276 - acc: 0.856 - 18s 736ms/step - loss: 0.3187 - acc: 0.8620 - val_loss: 0.3729 - val_acc: 0.8094
Epo

24/24 [==============================] - ETA: 11s - loss: 0.4626 - acc: 0.75 - ETA: 10s - loss: 0.4733 - acc: 0.71 - ETA: 7s - loss: 0.4810 - acc: 0.7292 - ETA: 8s - loss: 0.4430 - acc: 0.765 - ETA: 8s - loss: 0.4433 - acc: 0.762 - ETA: 7s - loss: 0.4084 - acc: 0.781 - ETA: 7s - loss: 0.3894 - acc: 0.794 - ETA: 7s - loss: 0.3676 - acc: 0.812 - ETA: 6s - loss: 0.3662 - acc: 0.812 - ETA: 6s - loss: 0.3611 - acc: 0.818 - ETA: 5s - loss: 0.3565 - acc: 0.818 - ETA: 5s - loss: 0.3732 - acc: 0.812 - ETA: 5s - loss: 0.3591 - acc: 0.822 - ETA: 4s - loss: 0.3629 - acc: 0.821 - ETA: 4s - loss: 0.3592 - acc: 0.825 - ETA: 3s - loss: 0.3613 - acc: 0.824 - ETA: 3s - loss: 0.3521 - acc: 0.830 - ETA: 2s - loss: 0.3414 - acc: 0.840 - ETA: 2s - loss: 0.3479 - acc: 0.832 - ETA: 1s - loss: 0.3590 - acc: 0.828 - ETA: 1s - loss: 0.3563 - acc: 0.830 - ETA: 0s - loss: 0.3521 - acc: 0.832 - ETA: 0s - loss: 0.3492 - acc: 0.834 - 14s 579ms/step - loss: 0.3447 - acc: 0.8387 - val_loss: 0.4015 - val_acc: 0.7969
Epo

24/24 [==============================] - ETA: 11s - loss: 0.5675 - acc: 0.62 - ETA: 10s - loss: 0.5459 - acc: 0.75 - ETA: 9s - loss: 0.5242 - acc: 0.7500 - ETA: 9s - loss: 0.5333 - acc: 0.718 - ETA: 8s - loss: 0.4754 - acc: 0.775 - ETA: 8s - loss: 0.4682 - acc: 0.781 - ETA: 8s - loss: 0.4427 - acc: 0.794 - ETA: 7s - loss: 0.4493 - acc: 0.789 - ETA: 7s - loss: 0.4517 - acc: 0.784 - ETA: 6s - loss: 0.4730 - acc: 0.768 - ETA: 6s - loss: 0.4962 - acc: 0.750 - ETA: 5s - loss: 0.4973 - acc: 0.755 - ETA: 5s - loss: 0.4819 - acc: 0.769 - ETA: 4s - loss: 0.4861 - acc: 0.763 - ETA: 4s - loss: 0.4942 - acc: 0.762 - ETA: 3s - loss: 0.4914 - acc: 0.761 - ETA: 3s - loss: 0.4829 - acc: 0.768 - ETA: 2s - loss: 0.4929 - acc: 0.760 - ETA: 2s - loss: 0.4871 - acc: 0.763 - ETA: 1s - loss: 0.4791 - acc: 0.768 - ETA: 1s - loss: 0.4718 - acc: 0.773 - ETA: 0s - loss: 0.4689 - acc: 0.775 - ETA: 0s - loss: 0.4718 - acc: 0.774 - 17s 724ms/step - loss: 0.4642 - acc: 0.7787 - val_loss: 0.4289 - val_acc: 0.8094
Epo

24/24 [==============================] - ETA: 11s - loss: 0.4072 - acc: 0.81 - ETA: 10s - loss: 0.4996 - acc: 0.81 - ETA: 10s - loss: 0.4916 - acc: 0.77 - ETA: 9s - loss: 0.4997 - acc: 0.7656 - ETA: 9s - loss: 0.4879 - acc: 0.775 - ETA: 8s - loss: 0.4625 - acc: 0.802 - ETA: 8s - loss: 0.4607 - acc: 0.803 - ETA: 7s - loss: 0.4738 - acc: 0.789 - ETA: 7s - loss: 0.4671 - acc: 0.791 - ETA: 6s - loss: 0.4471 - acc: 0.806 - ETA: 6s - loss: 0.4438 - acc: 0.812 - ETA: 5s - loss: 0.4273 - acc: 0.822 - ETA: 5s - loss: 0.4205 - acc: 0.826 - ETA: 4s - loss: 0.4162 - acc: 0.821 - ETA: 4s - loss: 0.4265 - acc: 0.816 - ETA: 3s - loss: 0.4402 - acc: 0.812 - ETA: 3s - loss: 0.4414 - acc: 0.812 - ETA: 2s - loss: 0.4295 - acc: 0.819 - ETA: 2s - loss: 0.4553 - acc: 0.805 - ETA: 1s - loss: 0.4610 - acc: 0.800 - ETA: 1s - loss: 0.4720 - acc: 0.788 - ETA: 0s - loss: 0.4764 - acc: 0.786 - ETA: 0s - loss: 0.4795 - acc: 0.788 - 14s 596ms/step - loss: 0.4680 - acc: 0.7966 - val_loss: 0.3991 - val_acc: 0.8313
Epo

24/24 [==============================] - ETA: 11s - loss: 0.2824 - acc: 0.93 - ETA: 10s - loss: 0.2635 - acc: 0.90 - ETA: 10s - loss: 0.3708 - acc: 0.87 - ETA: 9s - loss: 0.3930 - acc: 0.8594 - ETA: 9s - loss: 0.4576 - acc: 0.812 - ETA: 8s - loss: 0.4600 - acc: 0.812 - ETA: 8s - loss: 0.5114 - acc: 0.785 - ETA: 7s - loss: 0.4798 - acc: 0.796 - ETA: 7s - loss: 0.4972 - acc: 0.805 - ETA: 6s - loss: 0.4937 - acc: 0.800 - ETA: 6s - loss: 0.4748 - acc: 0.806 - ETA: 5s - loss: 0.4600 - acc: 0.812 - ETA: 5s - loss: 0.4535 - acc: 0.817 - ETA: 4s - loss: 0.4645 - acc: 0.812 - ETA: 4s - loss: 0.4728 - acc: 0.804 - ETA: 3s - loss: 0.4639 - acc: 0.804 - ETA: 3s - loss: 0.4568 - acc: 0.808 - ETA: 2s - loss: 0.4447 - acc: 0.816 - ETA: 2s - loss: 0.4350 - acc: 0.822 - ETA: 1s - loss: 0.4467 - acc: 0.815 - ETA: 1s - loss: 0.4341 - acc: 0.821 - ETA: 0s - loss: 0.4322 - acc: 0.821 - ETA: 0s - loss: 0.4230 - acc: 0.823 - 14s 592ms/step - loss: 0.4252 - acc: 0.8177 - val_loss: 0.3889 - val_acc: 0.8313
Epo

In [13]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('VGG_Mobilenet.csv', index=False)